# Softmax Classification

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

# Discrete Probability Distribution

$$P(class=i) = {e^i \over \sum e^i}$$

In [3]:
z = torch.FloatTensor([1,2,3])
print(z.size())

torch.Size([3])


### 수집 데이터를 기반으로 Maximum Likelihood Destination(MLD) 통해 훈련

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
hypothesis.sum()

tensor(1.)

# Cross Entropy

원하는 확률 분포 P와 Q가 얼마나 비슷한지를 알려주는 방법
$$H(P,Q) = -E_{x\sim P(X)} [log Q(x)] = -\sum_{x \in \mathcal{X}} P(x)log Q(x)$$

In [6]:
z = torch.rand(3,5,requires_grad=True)
print(z)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [7]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([0, 2, 1])


# Cross-Entropy Loss(Low-Level)
$$L= {1 \over N} \sum -y log(\hat{y})$$

In [8]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)
y_one_hot.scatter_(1, y.unsqueeze(1),1) #dim, index, source

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [9]:
y.unsqueeze(1)

tensor([[0],
        [2],
        [1]])

In [10]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean() #hypothesis:3x5 -> sum:3x1 -> mean:1
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [11]:
#y_one_hot.scatter_(1,y.unsqueeze(1), 3)
"""
tensor([[3., 0., 0., 0., 0.],
        [0., 0., 3., 0., 0.],
        [0., 3., 0., 0., 0.]])
"""

'\ntensor([[3., 0., 0., 0., 0.],\n        [0., 0., 3., 0., 0.],\n        [0., 3., 0., 0., 0.]])\n'

# Cross-Entropy Loss with torch.nn.functional

In [12]:
torch.log(F.softmax(z,dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [13]:
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [14]:
#Low Level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [15]:
#High Level
#NLL : Negative Log Likelihood
F.nll_loss(F.log_softmax(z, dim=1),y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [16]:
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward>)

# Tranining with Low-Level Cross Entropy Loss

In [17]:
x_train = [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
          [4,1,5,5],
          [1,7,5,5],
          [1,2,5,6],
          [1,6,6,6],
          [1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
print(x_train.size(), y_train.size())

torch.Size([8, 4]) torch.Size([8])


In [18]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    hypothesis = F.softmax(x_train.matmul(W)+b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1),1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


# F.cross_entropy

In [19]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    z = x_train.matmul(W)+b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


# High-Level Implementation with nn.Module

In [20]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
        
    def forward(self,x):
        return self.linear(x)        

In [21]:
model = SoftmaxClassifierModel()

In [22]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269521
Epoch 1000/1000 Cost: 0.241922
